# Multiple flows

1. Flow 1 runs, inputs bar1 and outputs bar2
2. Flow 2 runs, inputs Flow1.input, Flow1.output

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init(".")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

01-17 19:09:23 doltpy.core.dolt INFO     Initializing Dolt repo in existing dir .
01-17 19:09:23 doltpy.core.dolt INFO     Creating a new repo in .


DoltException: (['dolt', 'init'], b'', b'This directory has already been initialized.\n', 1)

In [7]:
dolt.log(number="1").popitem()[1].message

'Add rows to bar'

In [13]:
!poetry run python3 demo.py run

Metaflow 2.2.5.post20+gitfa65b10 executing MultiFlowDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-16 19:43:28.634 Workflow starting (run-id 1610855008628440):
2021-01-16 19:43:28.641 [1610855008628440/start/1 (pid 20636)] Task is starting.
2021-01-16 19:43:29.782 [1610855008628440/start/1 (pid 20636)] 01-16 19:43:29 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-16 19:43:29.810 [1610855008628440/start/1 (pid 20636)] 01-16 19:43:29 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-16 19:43:29.810 [1610855008628440/start/1 (pid 20636)] 01-16 19:43:29 doltpy.core.dolt INFO     Initializing Dolt repo in existing dir /Users/max-hoffman/Documents/sandbox/dolt/metaflow/dolt-demos/2-lineage-2-multiple-flows
2021-01-16 19:43:29.810 [1610855008628440/start/1 (pid 20636)] 01-16 19:43:29 doltpy.core.dolt INFO  

2021-01-16 19:43:30.200 [1610855008628440/start/1 (pid 20636)] Task failed.
2021-01-16 19:43:30.200 Workflow failed.
2021-01-16 19:43:30.201 Terminating 0 active tasks...
2021-01-16 19:43:30.201 Flushing logs...
    Step failure:
    Step start (task-id 1) failed.

01-16 19:43:30 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-16 19:43:30 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [14]:
meta_doltdb = Dolt(".")

01-16 21:04:48 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306


In [24]:
from doltpy.core.read import read_table_sql

flow_name = "MultiFlowDemo"
run_id = "1610857660139885"
table_name = "bar"
# get database and commit from metadata
filters = f"flow_name = \"{flow_name}\""
filters += f" AND run_id = \"{run_id}\""
# filters += f" AND kind = "read"
df = read_table_sql(meta_doltdb, f"SELECT `database`, `commit` FROM `metadata` WHERE {filters}")
print(df)
database = df.database.values[0]
commit = df.commit.values[0]
db = Dolt(database)
table = read_table_sql(db, f"SELECT * FROM `{table_name}` AS OF \"{commit}\"")


01-16 21:10:47 doltpy.core.dolt INFO     database,commit
foo,al0ehd852dev9q37lf4rbjtdkdjo4qln

01-16 21:10:47 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
01-16 21:10:47 doltpy.core.dolt INFO     index,A,B
0,1,1
1,1,1
2,1,1
3,2,2
4,2,2
5,2,2



  database                            commit
0      foo  al0ehd852dev9q37lf4rbjtdkdjo4qln


In [25]:
table

,index,A,B
0,0,1,1
1,1,1,1
2,2,1,1
3,3,2,2
4,4,2,2
5,5,2,2
